<a href="https://colab.research.google.com/github/kdemertzis/ML_BigData_CyberSecurity-test/blob/master/Project_Python/Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pycaret
#!pip install modin
#!pip install pyyaml==5.4.1
#!pip install -q gradio 

In [2]:
import pandas as pd
#import modin.pandas as pd
import pycaret
from pycaret.classification import *
import gradio as gr
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
def encode_permission(df): # Binning permission 
    df.permission = df.permission.fillna(-0.5)
    bins = (-1, 4.3, 4.5, 5.1, 5.5, 5.8, 6, 6.4, 7, 7.5, 7.9)
    categories = pd.cut(df.permission, bins, labels=False)
    df.permission = categories
    return df

def encode_access(df): # Binning access
    df.access = df.access.fillna(-0.5)
    bins = (-1, 2, 2.5, 2.8, 3, 3.3, 4, 4.4)
    categories = pd.cut(df.access, bins, labels=False)
    df.access = categories
    return df

def encode_activity(df): # Binning activity 
    df.activity = df.activity.fillna(-0.5)
    bins = (-1, 0, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6,5, 7)
    categories = pd.cut(df.activity, bins, labels=False)
    df.activity = categories
    return df

def encode_sys(df): # Binning sys
    df.sys = df.sys.fillna(-0.5)
    bins = (-1, 0, 1, 1.5, 2, 2.5)
    categories = pd.cut(df.sys, bins, labels=False)
    df.sys = categories
    return df

In [4]:
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/ML_BigData_CyberSecurity-test/master/Demo/android_mal.csv')

def transform_features(df):
    df = encode_permission(df)
    df = encode_access(df)
    #df = encode_activity(df)
    df = encode_sys(df)
    return df

train = data[['permission',	'access',	'sys', 'class']]
train = transform_features(train)


X_all = train.drop(['class'], axis=1)
y_all = train['class']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))

accuracy:  0.9333333333333333


In [5]:
data.describe()

,permission,access,activity,sys
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [6]:
def predict_class(permission,	access,	sys):
    df = pd.DataFrame.from_dict({'permission': [permission], 'access': [access], 'sys': [sys]})
    df = encode_permission(df)
    df = encode_access(df)
    #df = encode_activity(df)
    df = encode_sys(df)
    pred = clf.predict_proba(df)[0]
    return {'benign': pred[0], 'adware': pred[1], 'rootkit': pred[2]}

In [8]:
permission = gr.inputs.Slider(minimum=4.3, maximum=7.9, default=5.843333, label="Permission")
access = gr.inputs.Slider(minimum=2, maximum=4.4, default=3.054, label="Access")
#activity = gr.inputs.Slider(minimum=1, maximum=6.9, default=3.7586, label="Activity")
sys = gr.inputs.Slider(minimum=0.1, maximum=2.5, default=1.1986, label=" Sys")

gr.Interface(predict_class, [permission,	access,	sys], "label", live=True).launch();

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://37047.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
